##### GAN
<img src="https://s1.ax1x.com/2022/11/10/zpxQ0O.png" width = 80% hieht = 80% />

In [ ]:
#网络结构 生成器 判别器
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 1)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return nn.Sigmoid()(x)


class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.fc1 = nn.Linear(128, 1024)
        self.fc2 = nn.Linear(1024, 2048)
        self.fc3 = nn.Linear(2048, 784)
        self.activation = nn.ReLU()

def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.fc3(x)
    x = x.view(-1, 1, 28, 28)
    return nn.Tanh()(x)

In [31]:
import torch.nn as nn
import torch
import torch.optim
import torchvision
from torchvision.utils import save_image 
from torchvision.datasets import CIFAR10 
from torch.utils.data import DataLoader 
import torchvision.transforms as transforms 
from torch.autograd import Variable
import time 

In [34]:
# #读取数据集
dataset = CIFAR10(root = '/mnt/data/linzhipeng/data/', 
                 download = True, transform = transforms.ToTensor()) #下载数据集
print("download_ok")

dataloader = DataLoader(dataset, batch_size= 640, shuffle= False) #按批次读取数据(一批64张，总共有50000张，所以有50000/64=781批)，shuffle= True打乱数据


Files already downloaded and verified
download_ok


In [35]:
#构建鉴别网络
n_d_feature = 64 
n_channel = 3 
dnet = nn.Sequential(
        nn.Conv2d(n_channel, n_d_feature, kernel_size=4,
                 stride=2, padding=1),
        nn.LeakyReLU(0.2),
        nn.Conv2d(n_d_feature, 2 * n_d_feature, kernel_size=4,
                 stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_d_feature),
        nn.LeakyReLU(0.2),
        nn.Conv2d(2 * n_d_feature, 4 * n_d_feature, kernel_size=4,
                 stride=2, padding=1, bias=False),
        nn.BatchNorm2d(4 * n_d_feature),
        nn.LeakyReLU(0.2),
        nn.Conv2d(4 * n_d_feature, 1, kernel_size=4)).cuda()
print(dnet)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
)


In [36]:
#构建生成网络
latent_size = 64 
n_g_feature = 64 

gnet = nn.Sequential(
        nn.ConvTranspose2d(latent_size, 4 * n_g_feature, kernel_size=4,
                          bias=False),
        nn.BatchNorm2d(4 * n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(4 * n_g_feature, 2 * n_g_feature, kernel_size=4,
                          stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(2 * n_g_feature, n_g_feature, kernel_size=4,
                          stride=2, padding=1, bias=False),
        nn.BatchNorm2d(n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(n_g_feature, n_channel, kernel_size=4,
                          stride=2, padding=1),
        nn.Sigmoid()).cuda()
print(gnet)

Sequential(
  (0): ConvTranspose2d(64, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (10): Sigmoid()
)


In [37]:
#网络初始化
import torch.nn.init as init
def weights_init(m):
    if type(m) in [nn.ConvTranspose2d, nn.Conv2d]:
        init.xavier_normal_(m.weight)
    elif type(m) == nn.BatchNorm2d:
        init.normal_(m.weight, 1.0, 0.02)
        init.constant_(m.bias, 0)
        
gnet.apply(weights_init)
dnet.apply(weights_init)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
)

In [38]:
#载入gpu训练的预参数
# checkpoint_d = torch.load('/mnt/data/linzhipeng/data/path/D.pth')
# checkpoint_g = torch.load('/mnt/data/linzhipeng/data/path/G.pth')
# dnet.load_state_dict(checkpoint_d)
# gnet.load_state_dict(checkpoint_g)

In [39]:
#定义损失
criterion = nn.BCEWithLogitsLoss().cuda()
#定义优化器
goptimizer = torch.optim.Adam(gnet.parameters(),
                             lr=0.0002, betas=(0.5, 0.999))
doptimizer = torch.optim.Adam(dnet.parameters(),
                             lr=0.0002, betas=(0.5, 0.999))

In [40]:
#生成噪音数据，输入到G网络的数据
batch_size = 64
fixed_noise = torch.randn(batch_size, latent_size, 1, 1).cuda()
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [43]:
#开始训练
start = time.time() #开始时间

epoch_num = 5#共训练5个周期
for epoch in range(epoch_num):
    for batch_idx, data in enumerate(dataloader):
        real_images, _ = data
        batch_size = real_images.shape[0]
        #训练判别器D
        labels = torch.ones(batch_size)#真实数据的标签：1
        preds = dnet(Variable(real_images.type(Tensor))) #将真实数据喂给D网络
        outputs = preds.reshape(-1) #转换成未知行
        dloss_real = criterion(outputs, labels.type(Tensor))
        dmean_real = outputs.sigmoid().mean() #计算判别器将多少真数据判别为真，仅用于输出显示
        
        noises = torch.randn(batch_size, latent_size, 1, 1)
        fake_images = gnet(noises.type(Tensor)) #生成假数据
        labels = torch.zeros(batch_size)#生成假数据的标签：0
        fake = fake_images.detach() #类似于固定生成器参数
        preds = dnet(fake) #将假数据喂给判别器
        outputs = preds.reshape(-1)#转换成未知行
        dloss_fake = criterion(outputs.type(Tensor), labels.type(Tensor))
        dmean_fake = outputs.sigmoid().mean() #计算判别器将多少假数据判断为真，仅用于输出显示
        
        dloss = dloss_real + dloss_fake #总的鉴别器损失为两者之和
        dnet.zero_grad()
        dloss.backward()
        doptimizer.step()
        
        #训练生成器G
        labels = torch.ones(batch_size)#在训练生成器G时，希望生成器的标签为1
        preds = dnet(fake_images)#让假数据通过鉴别网络
        outputs = preds.reshape(-1)#转换成未知行
        gloss = criterion(outputs.type(Tensor), labels.type(Tensor))
        gmean_fake = outputs.sigmoid().mean() #计算判别器将多少假数据判断为真，仅用于输出显示
        
        gnet.zero_grad()
        gloss.backward()
        goptimizer.step()
        
        #输出本步训练结果
        print('[{}/{}]'.format(epoch, epoch_num) + '[{}/{}]'.format(batch_idx, len(dataloader)) +
             '鉴别器G损失:{:g} 生成器D损失：{:g}'.format(dloss, gloss) + 
             '真数据判真比例：{:g} 假数据判真比例：{:g}/{:g}'.format(dmean_real, dmean_fake, gmean_fake))
        # if batch_idx % 100 == 0:
        #     fake = gnet(fixed_noise) #噪声生成假数据
        #     path = './data_new/gpu{:02d}_batch{:03d}.png'.format(epoch, batch_idx)
        #     save_image(fake, path, normalize=False)
            
end = time.time()
print(end/60) #输出结束时间

[0/5][0/79]鉴别器G损失:0.9154 生成器D损失：2.31664真数据判真比例：0.575332 假数据判真比例：0.242973/0.126431
[0/5][1/79]鉴别器G损失:0.972138 生成器D损失：1.92216真数据判真比例：0.631121 假数据判真比例：0.340107/0.182284
[0/5][2/79]鉴别器G损失:1.02591 生成器D损失：2.60733真数据判真比例：0.675618 假数据判真比例：0.416835/0.09561
[0/5][3/79]鉴别器G损失:0.987931 生成器D损失：1.7396真数据判真比例：0.552129 假数据判真比例：0.262021/0.208066
[0/5][4/79]鉴别器G损失:0.990182 生成器D损失：2.82155真数据判真比例：0.723516 假数据判真比例：0.440978/0.0785217
[0/5][5/79]鉴别器G损失:0.716574 生成器D损失：2.22234真数据判真比例：0.648296 假数据判真比例：0.194993/0.135876
[0/5][6/79]鉴别器G损失:0.728722 生成器D损失：2.59689真数据判真比例：0.765243 假数据判真比例：0.33607/0.0952363
[0/5][7/79]鉴别器G损失:0.684753 生成器D损失：2.25645真数据判真比例：0.701562 假数据判真比例：0.242817/0.128819
[0/5][8/79]鉴别器G损失:0.808117 生成器D损失：2.67068真数据判真比例：0.745111 假数据判真比例：0.365402/0.0870135
[0/5][9/79]鉴别器G损失:0.883429 生成器D损失：2.00307真数据判真比例：0.637116 假数据判真比例：0.296125/0.163386
[0/5][10/79]鉴别器G损失:1.03869 生成器D损失：2.57886真数据判真比例：0.700489 假数据判真比例：0.443423/0.0968162
[0/5][11/79]鉴别器G损失:1.1394 生成器D损失：1.23688真数据判真比例：0.507275 假数据判真比例：0.271804/0.32

## 保存模型

In [44]:
torch.save(dnet.state_dict(),'/mnt/data/linzhipeng/data/path/D.pth')
torch.save(gnet.state_dict(),'/mnt/data/linzhipeng/data/path/G.pth')